In [10]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *

In [11]:
json_file_path = 'benchmarks/polyhedral/cases/case2.json'

data = json_file_to_isl(json_file_path)

In [12]:
domain = data["domain"]
access = data["read_dependencies"]
schedule = data["schedule"]
qops = data["Qops"]

In [13]:
physical_qubits_domain = isl.Set(f"{{ Q[i] : 1 <= i <= {qops} }}")

In [14]:
# backend

graph = defaultdict(list)
    
graph['Q[1]'] = ['Q[2]', 'Q[4]']
graph['Q[2]'] = ['Q[1]', 'Q[3]', 'Q[5]']
graph['Q[3]'] = ['Q[2]', 'Q[6]']
graph['Q[4]'] = ['Q[1]', 'Q[5]']
graph['Q[5]'] = ['Q[2]', 'Q[4]', 'Q[6]']
graph['Q[6]'] = ['Q[3]', 'Q[5]']


backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [15]:
initial_mapping = isl.Map(f"{{ q[i] -> Q[i] : 1<=i<={qops} }}")

In [16]:
programme = schedule.reverse().apply_range(access)

In [17]:
mapping = initial_mapping
swap_count = 0

while not programme.is_empty():
    programme_mapped = programme.apply_range(mapping)
    programme_access = programme_mapped.flat_range_product(programme_mapped).intersect_range(isl.Set("{ [i, j] : i > j }"))
    first_disconnected_edge = programme_access.apply_range(backend_disconnected_edges).lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge.domain()).range().as_set()
    first_qubit , second_qubit = disconected_qubits.lexmin().to_str().replace("{", "").replace("}", "").strip() , disconected_qubits.lexmax().to_str().replace("{", "").replace("}", "").strip()
    swap_count += shortest_paths[first_qubit]['costs'][second_qubit] -1
    new_domain = programme.domain().as_set().lex_ge_set(first_disconnected_edge.domain().as_set()).domain()
    programme = programme.intersect_domain(new_domain)

    swap_map = swaps_to_isl_map(shortest_paths[first_qubit]['paths'][second_qubit])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,initial_mapping,physical_qubits_domain)


In [18]:
swap_count

0